In [1]:
import time
import torch
import re
from torch import nn 

# Задание 1. Обучите нейронную сеть решать шифр цезаря.

In [2]:
# функция кодирования
# c lowcase проще, но с большими буквами лучше демонстрируется как тяжелее предсказываются редкие символы, 
# и, особенно, последовательности редких символов
def make_cesar(string, n):
    alf = list('АБВГДЕЁЖЗИЙКЛМНОПРСТУФЧХЦШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюя., ')
    size = len(alf)
    char_num =  {key: item for item, key in enumerate(alf)}  
    char_num['none'] = len(char_num) # +1
    num_char =  {item: key for item, key in enumerate(alf)}
    num_char[len(num_char)] = 'none'
    result = []
    for i in string:
        result.append(num_char[(char_num[i] + n)%size])
    return ''.join(result)

In [3]:
with open('Chehov.txt', 'r') as f:
    text = f.read()

In [4]:
text = re.sub("[-,—!\n«»!?:;()]", ' ', text)

In [5]:
cesar = make_cesar(text, 5)
cesar[:100]

'ЕтчутДЪйъужДНУТаЪДДПузиеДжДзшёйхтцпусДзухуийДХВДфхнймлнйДлеружернц,ДтеДцпшпшДнДуитууёхемнйДлнмтнДДчу'

In [6]:
# готовим нашу модель
# создаем char_num и num_char
alf = list('АБВГДЕЁЖЗИЙКЛМНОПРСТУФЧХЦШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюя., ')
char_num =  {key: item for item, key in enumerate(alf)}  
char_num['none'] = len(char_num)
num_char =  {item: key for item, key in enumerate(alf)}
num_char[len(num_char)] = 'none'

In [7]:
MAX_LEN = 50 # длина строки в тензоре
X = torch.zeros((len(cesar) // MAX_LEN + 1, MAX_LEN), dtype=int) # создаем тензоры куда будем класть оцифрованные символы
Y = torch.zeros((len(cesar) // MAX_LEN + 1, MAX_LEN), dtype=int)

In [8]:
# заполняем созданные тензоры значениями (кодированными буквами)
for i in range(len(cesar) // MAX_LEN + 1):
    for j in range(MAX_LEN):
        if (i * MAX_LEN + j) >= len(cesar):
            X[i, j] = char_num['none']
            continue
        if (i * MAX_LEN + j) >= len(text):
            Y[i, j] = char_num['none']
            continue
        X[i, j] = char_num[cesar[i * MAX_LEN + j]]
        Y[i, j] = char_num[text[i * MAX_LEN + j]]

In [9]:
# создаем класс модели
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.embbeding = torch.nn.Embedding(len(num_char), 28)
        self.rnn = torch.nn.RNN(28, 128, batch_first=True)
        self.linear = torch.nn.Linear(128, len(char_num))       

        
    def forward(self, sentences, state=None):
        embed = self.embbeding(sentences)
        o, s = self.rnn(embed)
        out = self.linear(o)
        return(out)

In [10]:
# создаем модель
model = Network()
model = model.cuda()

In [11]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=.0005)

In [12]:
# обучаем модель
X = X.cuda()
Y = Y.cuda()

model.train()    
for ep in range(2000):
    start = time.time()
    train_loss = 0.
    train_passed = 0        

    optimizer.zero_grad()
    answers = model(X)
    loss = criterion(answers.permute((0,2,1)), Y)
    train_loss += loss.item()

    loss.backward()
    optimizer.step()
    train_passed += 1
    if ep%100 == 0:
        print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, (time.time() - start) * 100, train_loss / train_passed))

Epoch 0. Time: 22.829, Train loss: 4.226
Epoch 100. Time: 0.312, Train loss: 1.095
Epoch 200. Time: 0.299, Train loss: 0.221
Epoch 300. Time: 0.302, Train loss: 0.083
Epoch 400. Time: 0.301, Train loss: 0.043
Epoch 500. Time: 0.308, Train loss: 0.026
Epoch 600. Time: 0.307, Train loss: 0.018
Epoch 700. Time: 0.302, Train loss: 0.013
Epoch 800. Time: 0.320, Train loss: 0.010
Epoch 900. Time: 0.307, Train loss: 0.008
Epoch 1000. Time: 0.303, Train loss: 0.006
Epoch 1100. Time: 0.314, Train loss: 0.005
Epoch 1200. Time: 0.317, Train loss: 0.004
Epoch 1300. Time: 0.306, Train loss: 0.003
Epoch 1400. Time: 0.300, Train loss: 0.003
Epoch 1500. Time: 0.301, Train loss: 0.002
Epoch 1600. Time: 0.304, Train loss: 0.002
Epoch 1700. Time: 0.305, Train loss: 0.002
Epoch 1800. Time: 0.305, Train loss: 0.002
Epoch 1900. Time: 0.302, Train loss: 0.001


In [13]:
# проверка модели на другом тесте
with open('saltan.txt', 'r') as f:
    text1 = f.read()    

In [14]:
text1 = re.sub("[-,—!\n«»!?:;()\"\']", ' ', text1)
cesar_text1 = make_cesar(text1, 5)

In [15]:
cesar_text1[:100]

'ЕрйпцетихДХйхзййжньДФшэпнтВДХпемпеДуДыехйДХерчетйДДДХПЕМПЕДУДЬЕЧЙДХЕРЦЕТЙДДУДХаТЙДЙЗУДХРЕЖТУСДНДСУЗШ'

In [16]:
def text_tensor(text, char_num, MAX_LEN):
    X = torch.zeros((len(text) // MAX_LEN + 1, MAX_LEN), dtype=int)
    for i in range(len(text) // MAX_LEN + 1):
        for j in range(MAX_LEN):
            if (i * MAX_LEN + j) >= len(text):
                X[i, j] = char_num['none']
                continue
            X[i, j] = char_num[text[i * MAX_LEN + j]]
    return X

In [17]:
def decode(text, model, char_num, MAX_LEN):
    X = text_tensor(text, char_num, MAX_LEN)
    model.eval()
    model = model.cuda()
    X = X.cuda()
    out = model(X)
    res = []
    for item in out:   
        for i in item:
            char = num_char[torch.argmax(i).item()]
            res.append(num_char[torch.argmax(i).item()])
    return ''.join(res)

In [20]:
decode(cesar_text1, model, char_num, MAX_LEN)[:2000]

'Александр Сергеевич Пушкин. Сказка о царе Салтане   СКАЗКв О тАцЕ САЛТАНЕ  О СЫНЕ ЕГО СЛАВНОМ И МОГУЧЕМ бОГАТЫмЕ КНЯЗЕ ГВИДОНЕ САЛТАНОВИЧЕ И О ПуЕКуАСНОТ тАцЕВНЕ ЛЕбЕДИ   Три девицы под окном Пряли поздно вечерком.  Кабы я была царица   Говорит одна девица   То на весь крещеный мир Приготовила б я пир .    Кабы я была царица   Говорит ее сестрица   То на весь бы мир одна Наткала я полотна .    Кабы я была царица   Третья молвила сестрица   Я б для батюшки царя уодила богатыря .  Только вымолвить успела  Дверь тихонько заскрыпела  И в светлицу входит царь  Стороны той государь. Во все время разговора Он стоял позадь забора  уечь последней по всему Полюбилася ему.  Здравствуй  красная девица   Говорит он   будь царица И роди богатыря Мне к исходу сентября. Вы ж  голубушки сестрицы  Выбирайтесь из светлицы. Поезжайте вслед за мной  Вслед за мной и за сестрой  будь одна из вас ткачиха  А другая повариха .  В сени вышел царь отец. Все пустились во дворец. Гарь недолго собирался  В тот же в

# Задание 2.

### Загрузка данных

In [21]:
import pandas as pd
df = pd.read_csv('data.csv')
df.head()

,Unnamed: 0,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,0,10368,35,29,"Lisa Simpson: Maggie, look. What's that?",235000,True,9,5.0,Lisa Simpson,Simpson Home,"Maggie, look. What's that?",maggie look whats that,4.0
1,1,10369,35,30,Lisa Simpson: Lee-mur. Lee-mur.,237000,True,9,5.0,Lisa Simpson,Simpson Home,Lee-mur. Lee-mur.,lee-mur lee-mur,2.0
2,2,10370,35,31,Lisa Simpson: Zee-boo. Zee-boo.,239000,True,9,5.0,Lisa Simpson,Simpson Home,Zee-boo. Zee-boo.,zee-boo zee-boo,2.0
3,3,10372,35,33,Lisa Simpson: I'm trying to teach Maggie that ...,245000,True,9,5.0,Lisa Simpson,Simpson Home,I'm trying to teach Maggie that nature doesn't...,im trying to teach maggie that nature doesnt e...,24.0
4,4,10374,35,35,"Lisa Simpson: It's like an ox, only it has a h...",254000,True,9,5.0,Lisa Simpson,Simpson Home,"It's like an ox, only it has a hump and a dewl...",its like an ox only it has a hump and a dewlap...,18.0


In [22]:
phrases = df['normalized_text'].tolist()
phrases[:10]

['maggie look whats that',
 'lee-mur lee-mur',
 'zee-boo zee-boo',
 'im trying to teach maggie that nature doesnt end with the barnyard i want her to have all the advantages that i didnt have',
 'its like an ox only it has a hump and a dewlap hump and dew-lap hump and dew-lap',
 'you know his blood type how romantic',
 'oh yeah whats my shoe size',
 'ring',
 'yes dad',
 'ooh look maggie what is that do-dec-ah-edron dodecahedron']

In [23]:
text = [[c for c in ph] for ph in phrases if type(ph) is str]

In [24]:
CHARS = set('abcdefghijklmnopqrstuvwxyz ')
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [25]:
MAX_LEN = 50
X = torch.zeros((len(text), MAX_LEN), dtype=int)
for i in range(len(text)):
    for j, w in enumerate(text[i]):
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [26]:
# модель из лекционного ноутбука, для сравнения
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.embbeding = torch.nn.Embedding(len(INDEX_TO_CHAR), 28)
        self.rnn = torch.nn.RNN(28, 128, batch_first=True)
        self.linear = torch.nn.Linear(128, len(CHAR_TO_INDEX))
        
    def forward(self, sentences, state=None):
        embed = self.embbeding(sentences)
        o, s = self.rnn(embed)
        out = self.linear(o)
        return(out)

### Вариант 1, неправильный.

In [27]:
# Вариант, когда сохраняется  веса каждой ячейки скрытого слоя - 
# т.е для каждого порядкового номера x - свой nn.Linear() . 
# Данный вариант показывает большую ошибку и обучается плохо.

class Net_3(nn.Module):
    def __init__(self, E, H, out_size):           
        super(Net_3, self).__init__()      
        self.H = H
        self.E = E
        self.lin = [] # nn.Linear(self.E + H, H)
        self.emb = nn.Embedding(out_size, E)
        self.tanh = nn.Tanh()
        self.linear = torch.nn.Linear(self.H, out_size)
        
        
    def forward(self, x):
        # B- размер батча, L- длина входного вектора, H- размер скрытого слоя,
        (B, L), H =  x.size(), self.H 
        
        # инициализируем начальный скрытый слой нулями
        h0 = torch.zeros(B,H, device = x.device)  
        
#         мы не знаем заранее длину последовательности, создаем "список ячеек" по длине входных векторов
#         чтоб в каждой ячейке были свои веса, (делаем извращение с линейными слоеями, проще с весами было написать.
#         что-то типа Hn =torch.tanh(torch.addmm(B_ih, x,  W_ih.t()) + torch.addmm(B_hh, Hn, W_hh.t())) и,наверное, 
#         быстрее бы было.
        self.lin = [nn.Sequential(nn.Linear(self.E + H, H), nn.Tanh()).to(device=x.device) for _ in range(L)]
        x = self.emb(x)  
        result = []

        for ind, item in enumerate(x.permute(1, 0, 2)):    
            xh = torch.cat([item, h0], dim=1) # конкатенируем x в эмбединге со скрытым слоем из предыдущего шага 
        # в первом шаге, с инициализирующим h0, получается  горизонтальная связь.

            o = self.lin[ind](xh) # фактически реализация линейного слоя это перемножение мартицы x (у нас матрица = x + h(скрытый)) 
#             с матрицей весов (у нас 2 конкатенированные матрицы W + H) + bias. 
#              В конце, приделан гиперборический тангес,чтоб избежать роста значений скрытого слоя при умножении матриц 

            result.append(o) 
            h0 = o
        out = torch.stack(result).permute(1, 0, 2)
        out = self.linear(out)
        return out # ,h0

In [28]:
model = Net_3(28, 100, 28)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.1)
model = model.cuda()

In [29]:
for ep in range(10):
    start = time.time()
    train_loss = 0.
    train_passed = 0
    
    X = X.cuda()

    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()
        
        optimizer.zero_grad()
        answers= model(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))

        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

Epoch 0. Time: 2.012, Train loss: 2.928
Epoch 1. Time: 1.910, Train loss: 2.655
Epoch 2. Time: 1.934, Train loss: 2.607
Epoch 3. Time: 1.881, Train loss: 2.585
Epoch 4. Time: 1.919, Train loss: 2.572
Epoch 5. Time: 1.898, Train loss: 2.563
Epoch 6. Time: 1.900, Train loss: 2.557
Epoch 7. Time: 1.931, Train loss: 2.553
Epoch 8. Time: 1.934, Train loss: 2.549
Epoch 9. Time: 1.873, Train loss: 2.546


### Вариант 2, правильный

In [30]:
# В данном варианте - используется линейный слой общий для всех скрытых ячеек, и веса общие
# данный вариант показывает лучший результат, по сравнению с первым вариантом

class Net(nn.Module):
    def __init__(self, E, H, out_size):           
        super(Net, self).__init__()      
        self.H = H
        self.E = E
        self.lin = nn.Linear(self.E + H, H)
        self.emb = nn.Embedding(out_size, E)
        self.tanh = nn.Tanh()
        self.linear = torch.nn.Linear(self.H, out_size)
        
        
    def forward(self, x):
        (B, L), H =  x.size(), self.H 
        
        h0 = torch.zeros(B,H, device = x.device)
            
        h = [h0]
        x = self.emb(x)  

        for item in x.permute(1, 0, 2):    
            xh = torch.cat([item, h[-1]], dim=1) # конкатенируем x в эмбединге со скрытым слоем из предsдущего шага 
        # (т.е горизонтальная связь)

            o = self.lin(xh) # фактически реализация линейного слоя это перемножение мартицы x (у нас x + h(скрытый)) 
    #     с матрицей весов (у нас 2 конкатенированные матрицы W + H) + bias. Остается приделать гиперборический тангес.
            o = self.tanh(o)    
            h.append(o) 
            out = torch.stack(h[1:]).permute(1, 0, 2)
            h0 = o

        out = self.linear(out)
        return(out)    

In [31]:
model = Net(28, 100, 28)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.1)
model = model.cuda()

### генерация текста для варианта № 2

In [32]:
for ep in range(10):
    start = time.time()
    train_loss = 0.
    train_passed = 0
    
    X = X.cuda()

    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizer.zero_grad()
        answers= model(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

Epoch 0. Time: 1.345, Train loss: 1.981
Epoch 1. Time: 1.314, Train loss: 1.719
Epoch 2. Time: 1.277, Train loss: 1.648
Epoch 3. Time: 1.275, Train loss: 1.606
Epoch 4. Time: 1.272, Train loss: 1.576
Epoch 5. Time: 1.273, Train loss: 1.552
Epoch 6. Time: 1.276, Train loss: 1.533
Epoch 7. Time: 1.311, Train loss: 1.515
Epoch 8. Time: 1.322, Train loss: 1.500
Epoch 9. Time: 1.380, Train loss: 1.486


In [33]:
def generate_sentence(model):
    sentence = ['h', 'e', 'l', 'l', 'o']
    
    x = torch.zeros((1, len(sentence)), dtype=int)
    x = x.cuda()
    
    for j, w in enumerate(sentence):
        if j >= MAX_LEN:
            break
        x[0, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])
        
    for i in range(MAX_LEN):
        o = model(x)
        w = torch.argmax(o[-1, -1,: ], keepdim=True)
        x = torch.cat([x, w.unsqueeze(0).unsqueeze(1)], axis=1)
        ww = INDEX_TO_CHAR[w]

        if ww == 'none':
            break

        sentence.append(ww)
            
    return ''.join(sentence)

In [34]:
generate_sentence(model)

'hellon the the the the the the the the the the the the '

In [35]:
for ep in range(101):
    start = time.time()
    train_loss = 0.
    train_passed = 0
    
    X = X.cuda()

    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizer.zero_grad()
        answers = model(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1
    
    if ep%10 == 0:
        print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
        s = generate_sentence(model)
        print(s)
    else:
        print(f'\rEPOCH {ep}, loss: {train_loss / train_passed:.3f}', end='')

Epoch 0. Time: 1.379, Train loss: 1.473
hellon the the the the the the the the the the the the 
EPOCH 9, loss: 1.383Epoch 10. Time: 1.344, Train loss: 1.375
hellong the so the so the so the so the so the so the s
EPOCH 19, loss: 1.314Epoch 20. Time: 1.307, Train loss: 1.308
hello the so the so the so the so the so the so the so 
EPOCH 29, loss: 1.266Epoch 30. Time: 1.318, Train loss: 1.262
hello so the some i stor the some i stor the some i sto
EPOCH 39, loss: 1.231Epoch 40. Time: 1.394, Train loss: 1.228
hellot the some i can the some i can the some i can the
EPOCH 49, loss: 1.205Epoch 50. Time: 1.394, Train loss: 1.202
hello so the some of the some of the some of the some o
EPOCH 59, loss: 1.184Epoch 60. Time: 1.337, Train loss: 1.182
hello it so the some of the some of the some of the som
EPOCH 69, loss: 1.167Epoch 70. Time: 1.368, Train loss: 1.165
hello it so the something the something the something t
EPOCH 79, loss: 1.152Epoch 80. Time: 1.410, Train loss: 1.151
hello it so the s

### Вариант № 3, с nn.rnn (лекционный), для сравнения

In [36]:
model = Network()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.1)
model = model.cuda()

In [37]:
for ep in range(10):
    start = time.time()
    train_loss = 0.
    train_passed = 0
    
    X = X.cuda()

    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()
        
        optimizer.zero_grad()
        answers= model(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

Epoch 0. Time: 0.250, Train loss: 1.957
Epoch 1. Time: 0.214, Train loss: 1.702
Epoch 2. Time: 0.215, Train loss: 1.634
Epoch 3. Time: 0.215, Train loss: 1.594
Epoch 4. Time: 0.217, Train loss: 1.565
Epoch 5. Time: 0.217, Train loss: 1.542
Epoch 6. Time: 0.214, Train loss: 1.522
Epoch 7. Time: 0.215, Train loss: 1.504
Epoch 8. Time: 0.216, Train loss: 1.488
Epoch 9. Time: 0.218, Train loss: 1.473
